# <p style="text-align: center;"> Variational Linear Systems Code </p> 
<p style="text-align: center;"> Ryan LaRose </p>

This notebook briefly demonstrates the current state of the Variational Linear Systems (VLS) code. All code is contained in `vls_pauli.py`, which defines a `PauliSystem` class.

In [1]:
# =======
# imports
# =======
import time

import numpy as np

from vls_pauli import PauliSystem
from cirq import ParamResolver, Symbol, ops, Circuit, LineQubit

# Creating a Linear System of Equations

A `PauliSystem` consists of a matrix of the form
\begin{equation}
A = \sum_{k = 1}^{K} c_k \sigma_k
\end{equation}
where $c_k$ are complex coefficients and $\sigma_k$ are strings of Pauli operators. In code, we represent the matrix $A$ as arrays of strings corresponding to Pauli operators. For example, to represent the Pauli operators
\begin{align}
\sigma_1 &= \sigma_I \otimes \sigma_x \otimes \sigma_Y \otimes \sigma_Z
\end{align}
we would write:

In [2]:
# specify the pauli operators of the matrix
Amat_ops = np.array([["I", "X", "Y", "Z"]])

Coefficients $c_k$ are stored similarly as arrays of complex values:

In [ ]:
# specify the coefficients of each term in the matrix
Amat_coeffs = np.array([1-0j])

Finally, the solution vector
\begin{equation}
|b\rangle = U |0\rangle
\end{equation}
is represented by the unitary $U$ that (efficiently) prepares $|b\rangle$ from the ground state. For example, the unitary $U$ could be
\begin{equation}
U = \sigma_I \otimes \sigma_X \otimes \sigma_Y \otimes \sigma_Z,
\end{equation}
which we would represent in code as:

In [ ]:
# specify the unitary that prepares the solution vector b
Umat_ops = np.array(["I", "X", "Y", "Z"])

To create `PauliSystem`, we can then simply feed in `Amat_coeffs`, `Amat_ops`, and `Umat_ops`.

In [ ]:
# create a linear system of equations
system = PauliSystem(Amat_coeffs, Amat_ops, Umat_ops)

# Working with a `PauliSystem`

The `PauliSystem` class can tell basic information about the system:

In [ ]:
print("Number of qubits in system:", system.num_qubits())
print("Size of matrix:", system.size())

To see the actual matrix representation of the system (in the computational basis), we can do:

In [ ]:
# get the matrix of the system
matrix = system.matrix()
print(matrix)

We can also see the solution vector $|b\rangle$ by doing:

In [ ]:
b = system.vector()
print(b)

# Creating an Ansatz

Initially, the `PauliSystem` ansatz for $V$ is an empty circuit:

In [ ]:
# print out the initial (empty) ansatz
system.ansatz

We are free to pick whatever ansatz we wish. Here, we will use a product of single qubit rotations.

In [ ]:
system.make_ansatz_circuit()
system.ansatz

This circuit contains 48 parameters (4 qubits x 2 "gates" / qubit x 6 parameters / gate). (Note that printing the circuit gets cut off in the notebook, scroll side to side to see the entire circuit.) For our simple example, we will chop off some of the gates to make the optimization easier:

In [ ]:
# remove some of the gates and print it out
system.ansatz = system.ansatz[:-13]
system.ansatz

# Computing the Cost

The local cost function is computed via the Hadamard Test. The local cost function can be written
\begin{equation}
C_1 = 1 - \frac{1}{n} \sum_{k = 1}^{K} \sum_{l \geq k}^{K} w_{k, l} c_k c_l^* \sum_{j = 1}^{n} \text{Re} \, \langle V_{k, l}^{(j)} \rangle
\end{equation}
where
\begin{equation}
\langle V_{k, l}^{(j)} \rangle := \langle0^{\otimes n}| V^\dagger A_k^\dagger U P_j U^\dagger A_l V |0^{\otimes n}\rangle
\end{equation}
and 
\begin{equation}
    w_{k, l} = \begin{cases}
    1 \qquad \text{if } k = l\\
    2 \qquad \text{otherwise}
    \end{cases} .
\end{equation}
Thus we have $n K^2$ different circuits to compute the cost. An example of one (the $k = 0$, $l = 0$, $j = 0$ term) is shown below:

In [ ]:
system.make_hadamard_test_circuit(system.ops[0], system.ops[0], 0, "real")

The circuit for computing the norm
\begin{equation}
\langle 0 | V^\dagger A_k^\dagger A_l V | 0 \rangle = \langle \psi | A_k^\dagger A_l | \psi \rangle 
\end{equation}
for the example $k = 0$, $l = 0$ is shown below:

In [ ]:
system.make_norm_circuit(system.ops[0], system.ops[0], "real")

To compute the cost, we can call `PauliSystem.cost` or `PauliSystem.eff_cost` (the latter exploits symmetries to compute the cost more efficiently) and pass in a set of angles to the ansatz gates:

In [ ]:
# =======================================
# compute the cost for some set of angles
# =======================================

# normalize the coefficients
system.normalize_coeffs()

# get some angles
angles = np.random.randn(18)

# compute the cost and time it
start = time.time()
cost = system.eff_cost(angles)
end = time.time() - start

# print out the results
print("Local cost C_1 =", cost)
print("Time to compute cost =", end, "seconds")

# Solving the System

To solve the system, we minimize the cost function. We'll do this below with the Powell optimization algorithm.

In [ ]:
# ===============================================
# minimize the cost (prints each cost evaluation)
# ===============================================
start = time.time()
out = system.solve(x0=angles, opt_method="Powell")
end = time.time() - start

In [ ]:
print("It took {} minutes to solve the system.".format(round(end / 60)))
print("Number of iterations of optimization method:", out["nit"])
print("Number of function evaluations:", out["nfev"])

# Comparing the Estimated and Exact Solutions

In [ ]:
# get the optimal angles
opt_angles = out["x"]

# evaluate the cost at the optimal angles found
system.eff_cost(opt_angles)

# get a param resolver
param_resolver = ParamResolver(
    {str(ii) : opt_angles[ii] for ii in range(len(opt_angles))}
)

sol_circ = system.ansatz.with_parameters_resolved_by(param_resolver)
sol_circ

In [ ]:
xhat = sol_circ.to_unitary_matrix()[:, 0]
bhat = np.dot(matrix, xhat)

In [ ]:
print(bhat)
print(b)

In [ ]:
print("overlap of computed and exact solution =", np.dot(b.conj().T, bhat))

# Future Work

* Normalize the cost to be between zero and one! (divide by N_Ax)
* Better optimization methods.
    * Optimize over a subset of the parameters at a time, then loop through (and reoptimize).
    * Add random gates using simulated annealing.
    * Compute all $nK^2$ circuits in parallel.
* Compute expectations of local observables at each cost iteration.
* Allow for arbitrary unitaries (not just Paulis)